This is the source code for **Homework #2**

## Step 1: Import libraries

In [85]:
import os
import pandas as pd
import json, time, urllib.parse
import requests

## Step 2: Getting the Article and Population Data

**A. Data Upload**

In [72]:
# Load dataset
df_politician = pd.read_csv('politicians_by_country_sept2022.csv')
df_population = pd.read_csv('population_by_country_2022.csv')

**B. Column renaming for consistency**

In [73]:
# Rename columns
df_politician.rename(columns = {'name': 'article'}, inplace = True)
df_population.rename(columns = {'Geography': 'country'}, inplace = True)

## Step 3: Handling Inconsistencies within Data

**A. Removing Duplicates**

In [74]:
# Review dataframe info
df_politician.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7584 entries, 0 to 7583
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   article  7584 non-null   object
 1   url      7584 non-null   object
 2   country  7584 non-null   object
dtypes: object(3)
memory usage: 177.9+ KB


In [75]:
# Check for duplicate articles at country level
df_politician[df_politician[['article','country']].duplicated()]

,article,url,country
6295,Abdirahman Aw Ali Farrah,https://en.wikipedia.org/wiki/Abdirahman_Aw_Al...,Somalia
6309,Ibrahim Megag Samatar,https://en.wikipedia.org/wiki/Ibrahim_Megag_Sa...,Somalia


In [76]:
# Check all possible entries of the duplicate records at country level
df_politician.loc[df_politician['article'].isin(['Abdirahman Aw Ali Farrah','Ibrahim Megag Samatar'])]

,article,url,country
6198,Abdirahman Aw Ali Farrah,https://en.wikipedia.org/wiki/Abdirahman_Aw_Al...,Somalia
6231,Ibrahim Megag Samatar,https://en.wikipedia.org/wiki/Ibrahim_Megag_Sa...,Somalia
6295,Abdirahman Aw Ali Farrah,https://en.wikipedia.org/wiki/Abdirahman_Aw_Al...,Somalia
6309,Ibrahim Megag Samatar,https://en.wikipedia.org/wiki/Ibrahim_Megag_Sa...,Somalia


In [77]:
# Remove duplicate articles at country level
df_politician.drop_duplicates(inplace = True)
df_politician.reset_index(drop = True, inplace = True)
df_politician.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7582 entries, 0 to 7581
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   article  7582 non-null   object
 1   url      7582 non-null   object
 2   country  7582 non-null   object
dtypes: object(3)
memory usage: 177.8+ KB


In [78]:
# Check for duplicate articles at world level
df_politician[df_politician[['article']].duplicated()]

,article,url,country
1566,Rudi Kolak,https://en.wikipedia.org/wiki/Rudi_Kolak,Croatia
1654,Count Wenzel Chotek of Chotkow and Wognin,https://en.wikipedia.org/wiki/Count_Wenzel_Cho...,Czechia
1669,Eduard Hedvicek,https://en.wikipedia.org/wiki/Eduard_Hedvicek,Czechia
1676,Konstantin Jireček,https://en.wikipedia.org/wiki/Konstantin_Jireček,Czechia
1680,Maximilian Ulrich von Kaunitz,https://en.wikipedia.org/wiki/Maximilian_Ulric...,Czechia
1711,"Leopold, Count von Thun und Hohenstein","https://en.wikipedia.org/wiki/Leopold,_Count_v...",Czechia
1914,Ibrahim Harun,https://en.wikipedia.org/wiki/Ibrahim_Harun,Ethiopia
2513,José Alejandro de Aycinena,https://en.wikipedia.org/wiki/José_Alejandro_d...,Guatemala
2659,José Francisco Barrundia,https://en.wikipedia.org/wiki/José_Francisco_B...,Honduras
3419,Luca Rovinalti,https://en.wikipedia.org/wiki/Luca_Rovinalti,Italy


**Note:** Reviewing a few of these articles, it is unclear if these politicians have actually pursued politics in 2 countries or are just descendent of one country and worker in another. We will continue to keep these 48 records for now assuming that these politicians have career in both the countries.

In [79]:
# Follow the same steps for population data
df_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                233 non-null    object 
 1   Population (millions)  233 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.8+ KB


In [80]:
# Check for duplicate geography records
df_population[df_population.duplicated()]

,country,Population (millions)


**B. Missing Values**

In [81]:
df_politician.isnull().sum()

article    0
url        0
country    0
dtype: int64

In [82]:
df_population.isnull().sum()

country                  0
Population (millions)    0
dtype: int64

**Observation:** There are no missing record values in both the tables.

**C. Creating Region Column in Population Data**

In [83]:
# Include a flag to identify rows that provide cumulative regional population counts
def myFunc(val):
    if val.isupper():
        return 1
    else:
        return 0
    
df_population['region_flag'] = df_population['country'].apply(myFunc)


# Creating region
df_population['region'] = None
for i in range(df_population.shape[0]):
    if df_population.loc[i, 'region_flag'] == 1:
        df_population.loc[i, 'region'] = df_population.loc[i, 'country']
    else:
        df_population.loc[i, 'region'] = df_population.loc[i-1, 'region']

        
# Rename columns
df_population['country'] = df_population['country'].str.lower()
df_population['region'] = df_population['region'].str.lower()

**D. Converting Population output format**

In [84]:
# Convert Population (millions) to literal (multiply by 10^6)
df_population['Population (millions)'] = df_population['Population (millions)'].apply(lambda x: x*pow(10, 6))

# Rename columns
df_population.rename(columns = {'Population (millions)': 'population'}, inplace = True)

df_population.head()

,country,population,region_flag,region
0,world,7.963000e+09,1,world
1,africa,1.419000e+09,1,africa
2,northern africa,2.510000e+08,1,northern africa
3,algeria,4.490000e+07,0,northern africa
4,egypt,1.035000e+08,0,northern africa


## Step 4: Getting Article Quality Predictions

In [87]:
# Making a page info request to get the current page revision

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': 'choubju1@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2022',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = df_politician.article.to_list()

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}

In [88]:
def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    # Make sure we have an article title
    if not article_title: return None
    
    request_template['titles'] = article_title
        
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [ ]:
# Make the page info request on all the article listed to get revision id

df_politician['revision_id'] = None
for idx, title in enumerate(ARTICLE_TITLES):
    info = request_pageinfo_per_article(ARTICLE_TITLES[idx])
    dct = info['query']['pages']
    pageid = ''.join(list(dct.keys()))
    revision_id = dct[pageid].get('lastrevid')
    df_politician.loc[(df_politician['article'] == title), 'revision_id'] = revision_id

In [30]:
# Make ORES Request

# The current ORES API endpoint
API_ORES_SCORE_ENDPOINT = "https://ores.wikimedia.org/v3"
# A template for mapping to the URL
API_ORES_SCORE_PARAMS = "/scores/{context}/{revid}/{model}"

# Use some delays so that we do not hammer the API with our requests
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': 'choubju1@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2022'
}

# A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
ARTICLE_REVISIONS = {df_politician2.loc[i,'name']:df_politician2.loc[i,'lastrevid'] for i in range(df_politician2.shape[0])}

# This template lists the basic parameters for making an ORES request
ORES_PARAMS_TEMPLATE = {
    "context": "enwiki",        # which WMF project for the specified revid
    "revid" : "",               # the revision to be scored - this will probably change each call
    "model": "articlequality"   # the AI/ML scoring model to apply to the reviewion
}

In [31]:
def request_ores_score_per_article(article_revid = None, 
                                   endpoint_url = API_ORES_SCORE_ENDPOINT, 
                                   endpoint_params = API_ORES_SCORE_PARAMS, 
                                   request_template = ORES_PARAMS_TEMPLATE,
                                   headers = REQUEST_HEADERS,
                                   features=False):
    # Make sure we have an article revision id
    if not article_revid: return None
    
    # set the revision id into the template
    request_template['revid'] = article_revid
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # the features used by the ML model can sometimes be returned as well as scores
    if features:
        request_url = request_url+"?features=true"
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [80]:
# Make the page info request on all the article listed to get revision id

df_politician['lastrevid'].fillna(value = 0, inplace = True)
df_politician['article_quality'] = None

for article, revision_id in ARTICLE_REVISIONS.items():
    if revision_id != 0:
        score = request_ores_score_per_article(ARTICLE_REVISIONS[article])
        article_quality = score['enwiki']['scores'][str(revision_id)]['articlequality']['score']['prediction']
        df_politician.loc[(df_politician['name'] == article), 'article_quality'] = article_quality

(1, 'Shahjahan Noori')
request complete
(2, 'Abdul Ghafar Lakanwal')
request complete
(3, 'Majah Ha Adrif')
request complete
(4, 'Haroon al-Afghani')
request complete
(5, 'Tayyab Agha')
request complete
(6, 'Ahmadullah Wasiq')
request complete
(7, 'Aziza Ahmadyar')
request complete
(8, 'Muqadasa Ahmadzai')
request complete
(9, 'Mohammad Sarwar Ahmedzai')
request complete
(10, 'Amir Muhammad Akhundzada')
request complete
(11, 'Nasrullah Baryalai Arsalai')
request complete
(12, 'Mohammad Asim Asim')
request complete
(13, 'Atiqullah Atifmal')
request complete
(14, 'Abdul Rahim Ayoubi')
request complete
(15, 'Alhaj Mutalib Baig')
request complete
(16, 'Ismael Balkhi')
request complete
(17, 'Abdul Baqi Turkistani')
request complete
(18, 'Mohammad Ghous Bashiri')
request complete
(19, 'Abas Basir')
request complete
(20, 'Jan Baz')
request complete
(21, 'Ahmad Behzad')
request complete
(22, 'Bashir Ahmad Bezan')
request complete
(23, 'Rafiullah Bidar')
request complete
(24, 'Mohammad Siddiq C

request complete
(195, 'Sejfi Vllamasi')
request complete
(196, 'Syrja Vlora')
request complete
(197, 'Visarion Xhuvani')
request complete
(198, 'Visar Ymeri')
request complete
(199, 'Abdyl Ypi')
request complete
(200, 'Xhezair Zaganjori')
request complete
(201, 'Menduh Zavalani')
request complete
(202, 'Said Abadou')
request complete
(203, 'Tahar Allan')
request complete
(204, 'Mohamed Seghir Babes')
request complete
(205, 'Djelloul Baghli')
request complete
(206, 'Noureddine Bahbouh')
request complete
(207, 'Lamine Bechichi')
request complete
(208, 'Bakhti Belaïb')
request complete
(209, 'Mohamed Belouizdad')
request complete
(210, 'Mourad Benachenou')
request complete
(211, 'Benaouda Hadj Hacène Bachterzi')
request complete
(212, 'Souad Bendjaballah')
request complete
(213, 'Mohammed Saleh Bendjelloul')
request complete
(214, 'Mohamed Bensalam')
request complete
(215, 'Said Boualam')
request complete
(216, 'Abdeslam Bouchouareb')
request complete
(217, 'Brahim Chibout')
request comp

request complete
(380, 'Davit Lokyan')
request complete
(381, 'Hrant Maloyan')
request complete
(382, 'Khachatur Malumian')
request complete
(383, 'Aram Manukyan')
request complete
(384, 'Gevorg Melik-Karagyozyan')
request complete
(385, 'Vahan Minakhoryan')
request complete
(386, 'Ruben Mirzakhanyan')
request complete
(387, 'Badal Muradyan')
request complete
(388, 'Gabriel Noradunkyan')
request complete
(389, 'Suren Papikyan')
request complete
(390, 'Paul Paul')
request complete
(391, 'Garo Paylan')
request complete
(392, 'Genrikh Poghosyan')
request complete
(393, 'Ishkhan Saghatelyan')
request complete
(394, 'Martiros Sarukhanyan')
request complete
(395, 'Vartkes Serengülian')
request complete
(396, 'Serop Zakaryan')
request complete
(397, 'Servitchen')
request complete
(398, 'Suren Shadunts')
request complete
(399, 'Kasbar Sinabian')
request complete
(400, 'Ohannes Tchekidjian')
request complete
(401, 'Hrayr Tovmasyan')
request complete
(402, 'Usuv Beg')
request complete
(403, 'Keg

request complete
(565, 'Jameel Humadan')
request complete
(566, 'Isa bin Ibrahim Al Khalifa')
request complete
(567, 'Fahmi al-Jowder')
request complete
(568, 'Saud Kanoo')
request complete
(569, 'Essam Khalaf')
request complete
(570, 'Abdullah bin Hamad bin Isa Al Khalifa')
request complete
(571, 'Ibrahim ibn Khalifa Al Khalifa')
request complete
(572, 'Mai bint Mohammed Al Khalifa')
request complete
(573, 'Khallfa bin Duaij Al Khalifa')
request complete
(574, 'Hamad Al Kooheji')
request complete
(575, 'Abdullatif Al-Mahmood')
request complete
(576, 'Khalid Al Maskati')
request complete
(577, 'Abdulhussain bin Ali Mirza')
request complete
(578, 'Mohammed Al-Buainain')
request complete
(579, 'Mohammed bin Khalifa bin Ahmed')
request complete
(580, 'Mohammed bin Khalifa bin Ali Al Khalifa')
request complete
(581, 'Abdulrahman al-Nuaimi')
request complete
(582, 'Ammar Qambar')
request complete
(583, 'Isa Qassim')
request complete
(584, 'Ahmed Al-Salloum')
request complete
(585, 'Salman b

request complete
(749, 'Milou Jeunehomme')
request complete
(750, 'André Kempinaire')
request complete
(751, 'Willy Kuijpers')
request complete
(752, 'Rachid Madrane')
request complete
(753, 'Luc Michel')
request complete
(754, 'Fientje Moerman')
request complete
(755, 'André Molitor')
request complete
(756, 'Jean Pede')
request complete
(757, 'Jean-Marie Piret')
request complete
(758, 'André-Eugène Pirson')
request complete
(759, "Philippe-Ambroise d'Olmen, Baron de Poederlé")
request complete
(760, 'Marjet Van Puymbroeck')
request complete
(761, 'Jean Richardot the Younger')
request complete
(762, 'Gustave Rolin-Jaequemyns')
request complete
(763, 'Hippolyte Rolin')
request complete
(764, 'Father Samuel')
request complete
(765, 'Louis-Joseph Seutin')
request complete
(766, 'Alfons Siffer')
request complete
(767, 'Cyrille Tahay')
request complete
(768, 'Jacques Timmermans')
request complete
(769, 'Robert Urbain')
request complete
(770, 'Jacques Van Gompel')
request complete
(771, 'Mar

...

request complete
(6751, 'Heinrich Escher')
request complete
(6752, 'Johann Rudolf Faesch')
request complete
(6753, 'Peter Falck')
request complete
(6754, 'Franz Rudolf Frisching')
request complete
(6755, 'Urs Glutz von Blotzheim')
request complete
(6756, 'Jürg Grossen')
request complete
(6757, 'Tim Guldimann')
request complete
(6758, 'Georg Friedrich Heilmann')
request complete
(6759, 'Thomas Helbling')
request complete
(6760, 'Eva Herzog')
request complete
(6761, 'David Hess (painter)')
request complete
(6762, 'Karl Otto Hunziker')
request complete
(6763, 'Beat Jans')
request complete
(6764, 'Walter Kälin')
request complete
(6765, 'Frédéric-César de La Harpe')
request complete
(6766, 'Georgie Lamon')
request complete
(6767, 'Paul Lüönd')
request complete
(6768, 'Ferdinand Masset')
request complete
(6769, 'Frank Melloul')
request complete
(6770, 'Guy Mettan')
request complete
(6771, 'Jakob Meyer zum Hasen')
request complete
(6772, 'Kurt Meyer (Lucerne)')
request complete
(6773, 'Sophie

request complete
(6935, 'Roland Kpotsra')
request complete
(6936, 'Issifou Tabiou Taffa')
request complete
(6937, 'Kodjo Amegnisso Tossou')
request complete
(6938, 'Mawupé Valentin Vovor')
request complete
(6939, 'Sani Yaya')
request complete
(6940, 'Enele Maʻafu')
request complete
(6941, "Viliami Tupoulahi Mailefihi Tuku'aho")
request complete
(6942, 'ʻUluvalu')
request complete
(6943, 'Henry Alcazar')
request complete
(6944, 'James Biggart')
request complete
(6945, 'Cyrus Prudhomme David')
request complete
(6946, 'William Demas')
request complete
(6947, 'Gypsy (calypsonian)')
request complete
(6948, 'Isaac Hope')
request complete
(6949, 'Martin Joseph')
request complete
(6950, 'Kenneth Lalla')
request complete
(6951, 'Emmanuel Mzumbo Lazare')
request complete
(6952, 'Randall Mitchell')
request complete
(6953, 'Arnold Piggott')
request complete
(6954, 'David Pitt, Baron Pitt of Hampstead')
request complete
(6955, 'Kevin Ramnarine')
request complete
(6956, 'Sarran Teelucksingh')
reques

request complete
(7125, 'Bosmic Otim')
request complete
(7126, 'Ricky Richard Anywar')
request complete
(7127, 'Ruth Molly Lematia')
request complete
(7128, 'Eric Sakwa')
request complete
(7129, 'Sitenda Sebalu')
request complete
(7130, 'Wilson Tumwine')
request complete
(7131, 'Sarah Nambozo Wekomba')
request complete
(7132, 'Oleksiy Arestovych')
request complete
(7133, 'Oleh Barna')
request complete
(7134, 'Alexander Barybin')
request complete
(7135, 'Andriy Beshta')
request complete
(7136, 'Yurii Biriukov (volunteer)')
request complete
(7137, 'Ivan Bisyuk')
request complete
(7138, 'Ihor Brusylo')
request complete
(7139, 'Maksym Buzhanskyi')
request complete
(7140, 'Vyacheslav Chaus')
request complete
(7141, 'Dmytriy Leonidovich Cherniavskiy')
request complete
(7142, 'Ihor Dolhov')
request complete
(7143, 'Anatoliy Fedorchuk')
request complete
(7144, 'Ivan Gel')
request complete
(7145, 'Izet Hdanov')
request complete
(7146, 'Valeriy Holovko')
request complete
(7147, 'Oleksandr Honcha

request complete
(7303, 'Marialbert Barrios')
request complete
(7304, 'Pedro Bastidas')
request complete
(7305, 'Alejandra Benítez')
request complete
(7306, 'José Brito (politician)')
request complete
(7307, 'Rubén González Cárdenas')
request complete
(7308, 'José Curiel')
request complete
(7309, 'Felix Díaz Ortega')
request complete
(7310, 'Vicente José Gregorio Díaz Silva')
request complete
(7311, 'Alí Domínguez')
request complete
(7312, 'Franklyn Duarte')
request complete
(7313, 'Alberto Franceschi')
request complete
(7314, 'Arnoldo Gabaldón')
request complete
(7315, 'Evangelina García Prince')
request complete
(7316, 'Vladimir Augusto Gessen Rodríguez')
request complete
(7317, 'César González Martínez')
request complete
(7318, 'Luis Graterol Caraballo')
request complete
(7319, 'Liborio Guarulla')
request complete
(7320, 'Maryann Hanson')
request complete
(7321, 'Reinaldo Iturriza')
request complete
(7322, 'Andrés Izarra')
request complete
(7323, 'José Bernabé Gutiérrez')
request co

request complete
(7483, 'Andre S Holland')
request complete
(7484, 'Byron Hove')
request complete
(7485, 'Tonderai Kasu')
request complete
(7486, 'Wilson Khumbula')
request complete
(7487, 'Lovemore Madhuku')
request complete
(7488, 'Sarah Mahoka')
request complete
(7489, 'Alfred Makwarimba')
request complete
(7490, 'Washington Malianga')
request complete
(7491, 'Elliot Manyika')
request complete
(7492, 'Linda Masarira')
request complete
(7493, 'Jevan Maseko')
request complete
(7494, 'Anxious Jongwe Masuka')
request complete
(7495, 'Lovemore Matombo')
request complete
(7496, 'Bright Matonga')
request complete
(7497, 'Isaac Matongo')
request complete
(7498, 'Obedingwa Mguni')
request complete
(7499, 'Promise Mkwananzi')
request complete
(7500, 'Max Mnkandla')
request complete
(7501, 'Bekithemba Mpofu')
request complete
(7502, 'Earnest Mudzengi')
request complete
(7503, 'Kisinoti Mukwazhe')
request complete
(7504, 'Edwin Mushoriwa')
request complete
(7505, 'Munacho Mutezo')
request compl

## Step 5: Combining the Datasets

In [129]:
# Creating region

df_population['region'] = None
for i in range(df_population.shape[0]):
    if df_population.loc[i, 'flag'] == 1:
        df_population.loc[i, 'region'] = df_population.loc[i, 'geography']
    else:
        df_population.loc[i, 'region'] = df_population.loc[i-1, 'region']

df_population['geography'] = df_population['geography'].str.lower()
df_population.rename(columns = {'geography': 'country'}, inplace = True)

# Merging datasets
df_merge = df_population.merge(df_politician, how = 'outer', on = 'country', indicator = True)
df_merge.drop(columns = 'url', inplace = True)

In [145]:
# Output list of countries with no match

mask1 = df_merge['flag'] != 1
mask2 = df_merge['_merge'] != 'both'
countries_list = df_merge.loc[(mask1 & mask2), ['country']].drop_duplicates()
countries_list.to_csv('wp_countries-no_match.txt', index=False, header = False)

In [153]:
# Consolidate remaining data into CSV

mask = df_merge['_merge'] == 'both'
df_merge2 = df_merge.loc[mask]
df_merge2.drop(columns = '_merge', inplace = True)
df_merge2.reset_index(drop = True, inplace = True)
df_merge2.rename(columns = {'name': 'article_title', 'lastrevid': 'revision_id'}, inplace = True)

columns = ['country', 'region', 'population (millions)', 'article_title', 'revision_id', 'article_quality']
df_merge2 = df_merge2[columns]
df_merge2.to_csv('wp_politicians_by_country.csv', index=False)

## Step 6: Analysis

In [197]:
# Total articles per capita by country
cols = ['country', 'region', 'population (millions)']
table1 = df_merge2.groupby(cols).agg({'article_title':'count'}).reset_index()
table1.rename(columns = {'article_title':'article_count'}, inplace = True)
table1['total_articles_percapita'] = table1['article_count'] / table1['population (millions)']
table1.head()

,country,region,population (millions),article_count,total_articles_percapita
0,afghanistan,SOUTH ASIA,41.1,118,2.871046
1,albania,SOUTHERN EUROPE,2.8,83,29.642857
2,algeria,NORTHERN AFRICA,44.9,34,0.757238
3,andorra,SOUTHERN EUROPE,0.1,10,100.000000
4,angola,MIDDLE AFRICA,35.6,42,1.179775


In [203]:
# Total high quality articles per capita by country
mask = df_merge2['article_quality'].isin(['FA', 'GA'])
cols = ['country', 'region', 'population (millions)']
table2 = df_merge2[mask].groupby(cols).agg({'article_title':'count'}).reset_index()
table2.rename(columns = {'article_title':'article_count'}, inplace = True)
table2['total_high_quality_articles_percapita'] = table2['article_count'] / table2['population (millions)']
table2.head()

,country,region,population (millions),article_count,total_high_quality_articles_percapita
0,afghanistan,SOUTH ASIA,41.1,6,0.145985
1,albania,SOUTHERN EUROPE,2.8,6,2.142857
2,andorra,SOUTHERN EUROPE,0.1,2,20.000000
3,armenia,WESTERN ASIA,3.0,1,0.333333
4,azerbaijan,WESTERN ASIA,10.2,1,0.098039


In [216]:
# Total articles per capita by region
table3 = df_merge2.groupby('region').agg({'population (millions)': 'sum', 'article_title':'count'}).reset_index()
table3.rename(columns = {'article_title':'article_count'}, inplace = True)
table3['total_articles_percapita'] = table3['article_count'] / table3['population (millions)']
table3

,region,population (millions),article_count,total_articles_percapita
0,CARIBBEAN,1239.5,201,0.162162
1,CENTRAL AMERICA,1755.7,195,0.111067
2,CENTRAL ASIA,1788.4,106,0.059271
3,EAST ASIA,21789.8,246,0.011290
4,EASTERN AFRICA,19032.8,648,0.034046
5,EASTERN EUROPE,37316.1,736,0.019723
6,MIDDLE AFRICA,7919.0,203,0.025635
7,NORTHERN AFRICA,7639.9,227,0.029712
8,NORTHERN EUROPE,1348.4,262,0.194304
9,OCEANIA,110.1,86,0.781108


In [218]:
# Total high quality articles per capita by country
mask = df_merge2['article_quality'].isin(['FA', 'GA'])

table4 = df_merge2[mask].groupby('region').agg({'population (millions)':'sum', 'article_title':'count'}).reset_index()
table4.rename(columns = {'article_title':'article_count'}, inplace = True)
table4['total_high_quality_articles_percapita'] = table4['article_count'] / table4['population (millions)']
table4

,region,population (millions),article_count,total_high_quality_articles_percapita
0,CARIBBEAN,89.6,8,0.089286
1,CENTRAL AMERICA,102.2,10,0.097847
2,CENTRAL ASIA,45.2,3,0.066372
3,EAST ASIA,946.7,16,0.016901
4,EASTERN AFRICA,653.4,15,0.022957
5,EASTERN EUROPE,2852.2,38,0.013323
6,MIDDLE AFRICA,43.9,5,0.113895
7,NORTHERN AFRICA,132.6,7,0.052790
8,NORTHERN EUROPE,46.6,8,0.171674
9,OCEANIA,9.3,2,0.215054


## Step 7: Results

In [207]:
## Top 10 countries by coverage: The 10 countries with the highest total articles per capita (in descending order) .

table1[['country','total_articles_percapita']].sort_values(by = 'total_articles_percapita', 
                                                           ascending = False).head(10).reset_index(drop = True)

,country,total_articles_percapita
0,monaco,inf
1,palau,inf
2,liechtenstein,inf
3,tuvalu,inf
4,san marino,inf
5,nauru,inf
6,antigua and barbuda,170.000000
7,federated states of micronesia,130.000000
8,andorra,100.000000
9,barbados,93.333333


In [208]:
## Bottom 10 countries by coverage: The 10 countries with the lowest total articles per capita (in ascending order) .

table1[['country','total_articles_percapita']].sort_values(by = 'total_articles_percapita', 
                                                           ascending = True).head(10).reset_index(drop = True)

,country,total_articles_percapita
0,china,0.001392
1,mexico,0.007843
2,saudi arabia,0.081744
3,romania,0.105263
4,india,0.126305
5,sri lanka,0.133929
6,egypt,0.135266
7,ethiopia,0.202593
8,taiwan,0.215517
9,vietnam,0.271630


In [209]:
## Top 10 countries by high quality: The 10 countries with the highest high quality articles per capita (in descending order).

table2[['country','total_high_quality_articles_percapita']].sort_values(by = 'total_high_quality_articles_percapita', 
                                                           ascending = False).head(10).reset_index(drop = True)

,country,total_high_quality_articles_percapita
0,tuvalu,inf
1,andorra,20.000000
2,montenegro,5.000000
3,albania,2.142857
4,suriname,1.666667
5,bosnia-herzegovina,1.470588
6,lithuania,1.071429
7,croatia,1.052632
8,slovenia,0.952381
9,palestinian territory,0.925926


In [210]:
## Bottom 10 countries by high quality: The 10 countries with the lowest high quality articles per capita (in ascending order).

table2[['country','total_high_quality_articles_percapita']].sort_values(by = 'total_high_quality_articles_percapita', 
                                                           ascending = True).head(10).reset_index(drop = True)

,country,total_high_quality_articles_percapita
0,india,0.004234
1,thailand,0.014970
2,japan,0.016013
3,nigeria,0.018307
4,vietnam,0.020121
5,colombia,0.020367
6,uganda,0.021186
7,pakistan,0.021204
8,sudan,0.021322
9,iran,0.022573


In [226]:
## Geographic regions by total coverage: A rank ordered list of geographic regions (in descending order) 
## by total articles per capita.

table3['rank'] = table3['total_articles_percapita'].rank(ascending = False)
table3.sort_values('total_articles_percapita', ascending = False).reset_index(drop = True)

,region,population (millions),article_count,total_articles_percapita,rank
0,OCEANIA,110.1,86,0.781108,1.0
1,NORTHERN EUROPE,1348.4,262,0.194304,2.0
2,CARIBBEAN,1239.5,201,0.162162,3.0
3,CENTRAL AMERICA,1755.7,195,0.111067,4.0
4,CENTRAL ASIA,1788.4,106,0.059271,5.0
5,SOUTHERN EUROPE,19444.6,890,0.045771,6.0
6,WESTERN ASIA,15577.1,687,0.044103,7.0
7,EASTERN AFRICA,19032.8,648,0.034046,8.0
8,NORTHERN AFRICA,7639.9,227,0.029712,9.0
9,MIDDLE AFRICA,7919.0,203,0.025635,10.0


In [227]:
## Geographic regions by high quality coverage: Rank ordered list of geographic regions (in descending order) 
## by high quality articles per capita.

table4['rank'] = table4['total_high_quality_articles_percapita'].rank(ascending = False)
table4.sort_values('total_high_quality_articles_percapita', ascending = False).reset_index(drop = True)

,region,population (millions),article_count,total_high_quality_articles_percapita,rank
0,OCEANIA,9.3,2,0.215054,1.0
1,NORTHERN EUROPE,46.6,8,0.171674,2.0
2,MIDDLE AFRICA,43.9,5,0.113895,3.0
3,CENTRAL AMERICA,102.2,10,0.097847,4.0
4,CARIBBEAN,89.6,8,0.089286,5.0
5,CENTRAL ASIA,45.2,3,0.066372,6.0
6,NORTHERN AFRICA,132.6,7,0.052790,7.0
7,WESTERN ASIA,538.8,28,0.051967,8.0
8,SOUTHERN EUROPE,910.6,46,0.050516,9.0
9,SOUTH AMERICA,302.3,13,0.043004,10.0
